# Xử lí dữ liệu thiếu

Sau khi đã làm sạch, chuẩn hóa định dạng cần tiến hành điền dữ liệu bị thiếu trước khi encoding

# 0. Nguồn dữ liệu

In [ ]:
import pandas as pd
import numpy as np
import unidecode
from sklearn.model_selection import KFold

df = pd.read_csv('../data/interim/normalize_interim.csv')

# Lưu thay đổi

In [ ]:
csv_file = '../data/interim/fill_interim.csv'